In [3]:
!pip install torch-snippets

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
!cp /content/drive/Shareddrives/MO434/dataset/project_dataset_corel_ct.zip .
!unzip -q project_dataset_corel_ct.zip

In [5]:
from torch_snippets import *
import torch
import sys


batchsize = 32
n_fold = 3
input_shape = (3,224,224)
device = torch.device('cuda')
test_only = False

lr = 1e-6
weight_decay=1e-3
n_epochs = 50

scheduler_step=20
scheduler_gamma = 0.5

dataset_dir = "/content/project_dataset_corel"
#dataset_dir = "../dataset/project_dataset_corel"

log_dir='/content/drive/Shareddrives/MO434/executions/ct/ct_freeze'
#log_dir='./log'

def create_dir_if_necessary(dir):
    try:
        os.makedirs(dir)
    except FileExistsError:
        pass
create_dir_if_necessary(log_dir)

[07/05/23 21:35:49] WARNING  Unable to load torch and dependent libraries from                ]8;id=598121;file:///usr/local/lib/python3.10/dist-packages/torch_snippets/loader.py\loader.py]8;;\:]8;id=984820;file:///usr/local/lib/python3.10/dist-packages/torch_snippets/loader.py#<module>:98\<module>:98]8;;\
                             torch-snippets.                                                                       
                             Functionalities might be limited. pip install lovely-tensors in                       
                             case there are torch related errors                                                   

In [6]:
import pandas as pd
import typing
import os.path as osp
from PIL import Image

class CtDataset():
    def __init__(self, fold_dir, mode: typing.Literal['test', 'val', 'train'], transforms=None) -> None:
        self.fold_dir = fold_dir
        self.mode = mode
        self.transforms = transforms

        self.img_dir = osp.join(fold_dir, mode)
        self.pair_df = pd.read_csv(osp.join(fold_dir, f'ct_{self.mode}.csv'), header=None)

        self.img_dict = {}
        img_list = pd.concat((self.pair_df[0], self.pair_df[1]), ignore_index=True).drop_duplicates().reset_index(drop=True)

        for idx, item in img_list.items():
            img_path = osp.join(self.img_dir, item)
            img = Image.open(img_path)
            self.img_dict[item] = img

    def __len__(self):
        return len(self.pair_df)

    def __getitem__(self, idx):
        pair = self.pair_df.loc[idx, :]
        img1 = self.img_dict[pair[0]]
        img2 = self.img_dict[pair[1]]
        if self.transforms:
            img1 = self.transforms(img1)
            img2 = self.transforms(img2)
        return img1, img2, np.array([pair[2]])



In [7]:
from torchvision import transforms

class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean

    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean

    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomAffine(degrees=10, translate=(0.05,0.10), scale=(0.9,1.1), shear=(-2,2),
                            interpolation=transforms.InterpolationMode.BILINEAR,
                            fill=0),
    transforms.Resize((224,224), interpolation=transforms.InterpolationMode.BILINEAR,
                      max_size=None, antialias=True),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    #AddGaussianNoise(0, 0.1)
])

val_transforms = transforms.Compose([
    transforms.Resize((224,224), interpolation=transforms.InterpolationMode.BILINEAR,
                      max_size=None, antialias=True),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [8]:
import torch
from torch import nn as nn
import math
from collections import OrderedDict


class CMKNet(nn.Module):
    def __init__(self, dropout=0) -> None:
        super().__init__()

        self.dropout = dropout

        # input: 3, 224, 224
        self.col1 = nn.Sequential(
            self._conv_seq(3, 32, 3), # 32, 112, 112
            self._conv_seq(32, 64, 3), # 64, 56, 56
            self._conv_seq(64, 128, 3), # 128, 28, 28
        )

        self.dense = nn.Sequential(
            self._linear_seq(128*28*28, 1024, dropout),
            self._linear_seq(1024, 256, dropout),
            nn.Linear(256, 64)
        )

        self._initialize_weights()

        self.cam = False
        self.cam_grad = None

    def forward(self, x):
        x = self.col1(x)
        x = torch.flatten(x, 1)
        x = self.dense(x)

        return x

    def _register_grad(self, grad):
        self.cam_grad = grad

    def get_conv_gradient(self):
        return self.cam_grad

    def get_conv_activation(self, x):
        return self.col1(x)

    def get_output_per_layer(self, x):
        output_by_layer = OrderedDict()

        output_by_layer['input'] = x.clone().detach().cpu().data.numpy()

        x = self.col1(x)
        output_by_layer["featextract-conv"] = x.clone().detach().cpu().numpy()

        x = torch.flatten(x, 1)
        for layer_name, layer in self.dense.named_children():
            #do forward through the layer
            x = layer.forward(x)
            #save the output
            output_by_layer["classifier-"+layer_name] = x.clone().detach().cpu().numpy()

        return output_by_layer


    def _fuse(self, in_channels, out_channels, kernel_size):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, padding=kernel_size//2, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )

    def _linear_seq(self, in_features, out_features, dropout):
        return nn.Sequential(
            nn.Linear(in_features, out_features),
            nn.ReLU(),
            nn.Dropout(dropout)
        )

    def _conv_seq(self, in_channels, out_channels, kernel_size):
        "conv2d, batchnorm, relu, max pool"
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, padding=kernel_size//2, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

    def _initialize_weights(self):
        #for each submodule of our network
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                #get the number of elements in the layer weights
                n = m.kernel_size[0] * m.kernel_size[1] * m.in_channels
                #initialize layer weights with random values generated from a normal
                #distribution with mean = 0 and std = sqrt(2. / n))
                m.weight.data.normal_(mean=0, std=math.sqrt(2. / n))

                if m.bias is not None:
                    #initialize bias with 0
                    m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                #initialize layer weights with random values generated from a normal
                #distribution with mean = 0 and std = 1/100
                m.weight.data.normal_(mean=0, std=0.01)
                if m.bias is not None:
                #initialize bias with 0
                    m.bias.data.zero_()

def get_model_ct(params_dir=None):
    model = CMKNet()
    if params_dir:
        model.load_state_dict(torch.load(params_dir, map_location=device))
    return model

In [9]:
contrastive_thres = 1.1

class ContrastiveLoss(torch.nn.Module):
    """
    Contrastive loss function.
    Based on: http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    """

    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin
    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2, keepdim = True)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2)/2 +
            (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))/2
        #print(list(zip(euclidean_distance.cpu().detach().numpy(), label.cpu().detach().numpy())))
        #print(loss_contrastive)
        acc = ((euclidean_distance > contrastive_thres) == label).float().mean()
        return loss_contrastive, acc

In [10]:
import matplotlib.pyplot as plt

def dist_hist(best_model_path, trn_dl):
    # Distance histograms for the cases we have the same and different persons, respectively.
    model = get_model_ct(best_model_path).to(device)
    different = []
    same      = []
    model.eval()

    for ix, data in enumerate(trn_dl):
        if (ix + 1)%100 == 0:
            print('Processing batch {}/{}'.format((ix + 1), len(trn_dl)))
        imgsA, imgsB, labels = [t.to(device) for t in data]
        codesA, codesB       = model(imgsA), model(imgsB)
        labels               = labels.cpu()

        with torch.no_grad():
            euclidean_distance        = F.pairwise_distance(codesA, codesB)
            euclidean_distance        = euclidean_distance.cpu()

        for i in range(len(labels)):
            if (labels[i] == 0): # same person
                same.append(euclidean_distance[i].item())
            else:
                different.append(euclidean_distance[i].item())

    fig, ax = plt.subplots(figsize = (10,5))
    ax.set_title('Distance histograms for genuine and impostor comparisons', fontsize = 20, fontweight = 'bold')
    ax.set_xlabel('Euclidean Distance', fontsize = 16, fontweight = 'bold')
    ax.set_ylabel('Density', fontsize = 16, fontweight = 'bold')
    ax.hist(different,bins = 50, density=True, alpha = 0.7, label = 'different')
    ax.hist(same, bins = 50, density=True, alpha = 0.7, label = 'same')
    ax.tick_params(labelsize = 16, axis = 'both')
    ax.legend()
    ax.grid(True)
    plt.plot()

def train_batch_ct(model, data, optimizer, criterion):
    imgsA, imgsB, labels = [t.to(device) for t in data]
    optimizer.zero_grad()
    codesA, codesB = model(imgsA), model(imgsB)
    loss, acc = criterion(codesA, codesB, labels)
    loss.backward()
    optimizer.step()
    return loss.item(), acc.item()

@torch.no_grad()
def validate_ct(model, dataloder, criterion):
    model.eval()
    ac_acc, ac_loss = 0, 0
    cnt = 0
    for idx, data in enumerate(dataloder):
        imgsA, imgsB, labels = [t.to(device) for t in data]
        codesA, codesB = model(imgsA), model(imgsB)
        loss, acc = criterion(codesA, codesB, labels)
        ac_loss += loss
        ac_acc += acc
        cnt +=1
    model.train()
    return ac_loss.item()/cnt, ac_acc.item()/cnt

In [11]:

def train_ct(dataset_dir, best_model_path):
    # DataLoaders
    train_set = CtDataset(dataset_dir, 'train', transforms=train_transforms)
    val_set = CtDataset(dataset_dir, 'val', transforms=val_transforms)

    trn_dl = DataLoader(train_set, batch_size=batchsize, shuffle=True)
    val_dl = DataLoader(val_set, batch_size=1, shuffle=False)

    model = get_model_ct().to(device)
    model.train()

    criterion = ContrastiveLoss()
    optimizer = optim.Adam(model.parameters(),lr = lr, weight_decay=weight_decay)

    log = Report(n_epochs)

    best_model = None
    best_model_loss = sys.maxsize

    for epoch in range(n_epochs):
        N = len(trn_dl)
        epoch_train_loss = 0
        for i, data in enumerate(trn_dl):
            loss, acc = train_batch_ct(model, data, optimizer, criterion)
            epoch_train_loss += loss

        loss, acc = validate_ct(model, val_dl, criterion)
        if loss < best_model_loss:
            torch.save(model.state_dict(), best_model_path)
            best_model_loss = loss
            best_model = epoch

        log.record(epoch+1, train_loss=epoch_train_loss/N, val_loss=loss, val_acc=acc,
        best_model=best_model+1, best_loss=best_model_loss, end='\r')

    log.plot_epochs(['train_loss','val_loss'])
    log.plot_epochs(['val_acc'])
    del model
    torch.cuda.empty_cache()
    dist_hist(best_model_path, trn_dl)

In [ ]:
for fold in range(n_fold):
    fold_log_dir = osp.join(log_dir, f'fold{fold}')
    create_dir_if_necessary(fold_log_dir)

    train_ct(osp.join(dataset_dir, f'fold{fold}'), osp.join(fold_log_dir, 'best_model_ct.pth'))


# necessario remover dropout

EPOCH: 4.000  train_loss: 0.251  val_loss: 0.274  val_acc: 0.711  best_model: 4.000  best_loss: 0.274  (72.00s - 827.98s remaining)